In [26]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=6, linewidth=120)

So, recently, there has been a lot of research in our group concerned with **CUHF**: 'constrained' UHF. This is trying to modify UHF SCF with a constraint such that it should convergence to ROHF.

Since the nature of the proposed algorithms is inherently related to _modifying_ an _existing_ UHF SCF algorithm (plain, DIIS, ...) but with a small extra step or two, I think this is an excellent showcase for `GQCP`'s 'injection' capabilities`.

## Molecular setup

Let us start by defining the molecular system under consideration. We'll take an H3-ring, 1 a.u. apart and in the STO-3G basisset.

In [27]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h3.xyz" , 0)  # create a neutral molecule
N = molecule.numberOfElectrons()
N_P = molecule.numberOfElectronPairs()  # rounds down
N_alpha = N_P + 1
N_beta = N_P

print(molecule)
print("Number of alpha electrons: {}".format(N_alpha))
print("Number of beta electrons: {}".format(N_beta))

Number of electrons: 3 
H  (0, 0, 0)
H  (0, 1.63655, 0.944863)
H  (0, 0, 1.88973)

Number of alpha electrons: 2
Number of beta electrons: 1


In order to start any calculations, we need to calculate the molecular integrals. In `GQCP`, this is done by creating a restricted spin-orbital basis:

In [28]:
spinor_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")  # in AO basis; atomic spin-orbitals
K = spinor_basis.numberOfSpinors()

S = spinor_basis.quantize(gqcpy.OverlapOperator())

Then, we will quantize the molecular Hamiltonian (in which the interactions terms are the electron's kinetic energy, the nucleus-electron attraction and the interelectronic repulson) in that spin-orbital basis:

In [29]:
fq_hamiltonian = gqcpy.FQMolecularHamiltonian(molecule)
sq_hamiltonian = spinor_basis.quantize(fq_hamiltonian)

## The standard 'plain' UHF SCF algorithm

Let's establish a base line for the algorithms by examining the standard 'plain' UHF SCF algorithm. In `GQCP`, we have implemented algorithms and environments:

- an _environment_ acts as a 'calculation space', storing intermediate calculations;
- an _algorithm_ and its composing _steps_ continuously modify the environment.

For UHF SCF, we use an `UHFSCFEnvironment`:

In [30]:
environment = gqcpy.UHFSCFEnvironment_d.WithCoreGuess(N_alpha, N_beta, sq_hamiltonian, S)

where we have initialized the alpha- and beta-coefficient matrices with the (generalized) eigenvectors of the core Hamiltonian (in AO basis).

Let's now take a look at the plain UHF SCF solver:

In [31]:
plain_algorithm = gqcpy.UHFSCFSolver_d.Plain(threshold=1.0e-04, maximum_number_of_iterations=1000)

print(plain_algorithm.description())

An iterative algorithm (with a maximum of 1000 iterations) consisting of the following steps:
An algorithmic step consisting of 4 algorithmic steps:
	1. Calculate the current UHF alpha and beta density matrices (in the AO basis) and place them in the environment.
	2. Calculate the current UHF Fock matrices (expressed in the scalar/AO basis) and place them in the environment.
	3. Solve the generalized eigenvalue problem for the most recent scalar/AO Fock matrices. Add the associated coefficient matrices and orbital energies to the environment.
	4. Calculate the current electronic UHF energy and place it in the environment.

With the following convergence criterion:
A convergence criterion that checks if the norm of the difference of two iterates (the UHF spin resolved density matrix in AO basis) is converged, with a tolerance of 1.00e-04.


With this algorithm, let's check the UHF SCF procedure.

In [32]:
uhf_qc_structure = gqcpy.UHF_d.optimize(plain_algorithm, environment)

In [33]:
uhf_energy = uhf_qc_structure.groundStateEnergy()
uhf_parameters = uhf_qc_structure.groundStateParameters()

In [34]:
print("UHF electronic energy: {:.8f}\n".format(uhf_energy))
print("UHF C_alpha:\n{}\n".format(uhf_parameters.expansion().alpha.matrix()))
print("UHF C_beta:\n{}\n".format(uhf_parameters.expansion().beta.matrix()))

UHF electronic energy: -2.92351168

UHF C_alpha:
[[-4.600360e-01 -9.965131e-01  5.353575e-01]
 [-3.016595e-01  2.493687e-05 -1.183335e+00]
 [-4.600317e-01  9.964931e-01  5.353984e-01]]

UHF C_beta:
[[ 2.806909e-01 -6.476418e-01 -9.965199e-01]
 [ 6.388130e-01  1.040768e+00  2.216669e-05]
 [ 2.806998e-01 -6.476897e-01  9.964863e-01]]



## The CUHF (Scuseria/thesis) algorithm

One of the CUHF algorithms described by Scuseria's group can be found in the PhD thesis by T. Tsuchimochi. Since this algorithm uses a modification of the UHF alpha- and beta-Fock matrices, this is an ideal opportunity to use `GQCP`'s injection capabilities.

I've taken the reference implementation from Xeno's [ghf](https://github.com/GQCG/ghf) repository, and adapted it to `GQCP`.

We start off by defining a `Python` function that modifies the alpha- and beta- Fock matrices in an `UHFSCFEnvironment`. We then wrap that `Python` function into a special class `FunctionalStep_UHFSCFEnvironment`, which allows for the bridging between `Python` functions and `GQCP`'s Algorithm framework.

In [35]:
def constrain_function_thesis(environment):
    """Replace the alpha- and beta- UHF Fock matrices by their constrained counterparts, as explained in the CUHF 'thesis' algorithm."""

    # Prepare some variables to be used in the algorithm.
    X = spinor_basis.lowdinOrthonormalization().alpha.matrix()  # the transformation matrix TO an orthonormal basis
    
    F_alpha = environment.fock_matrices[-1].alpha.parameters()  # in AO basis
    F_beta = environment.fock_matrices[-1].beta.parameters()  # in AO basis
    
    D_alpha = environment.density_matrices[-1].alpha.matrix()  # in AO basis
    D_beta = environment.density_matrices[-1].beta.matrix()  # in AO basis
    
    N_alpha = environment.N.alpha
    N_beta = environment.N.beta
    
    # Form the closed-shell Fock matrix and the UHF modification.
    F_cs = (F_alpha + F_beta) / 2.0  # the closed-shell Fock matrix, in AO basis
    Delta_UHF = (F_alpha - F_beta) / 2.0  # in AO basis

    # Form modified Fock matrices.
    F_aa = F_cs + Delta_UHF
    F_bb = F_cs - Delta_UHF
    
    # Find the natural occupation numbers and vectors by diagonalizing the charge-density matrix in an orthonormal basis.
    P_AO = (D_alpha + D_beta) / 2.0
    P_MO = np.linalg.inv(X) @ P_AO @ np.linalg.inv(X.T)
    
    natural_occupation_numbers, V = np.linalg.eigh(P_MO)
    
    # Construct the Langrange multipliers to add them to the 'constrained' Fock matrices.
    Delta_UHF_NO = np.linalg.inv(V) @ np.linalg.inv(X) @ Delta_UHF @ np.linalg.inv(X.T) @ np.linalg.inv(V.T)  # in the natural occupation (NO) basis
    Lambda_NO = np.zeros(np.shape(Delta_UHF_NO))
    Lambda_NO[:N_beta, N_alpha:] = -Delta_UHF_NO[:N_beta, N_alpha:]
    Lambda_NO[N_alpha:, :N_beta] = -Delta_UHF_NO[N_alpha:, :N_beta]
    Lambda_AO = X @ V @ Lambda_NO @ V.T @ X.T
    

    # Overwrite the most recent UHF Fock matrices with the cUHF ones
    
    F_alpha_constrained = F_aa + Lambda_AO
    #environment.replace_current_fock_matrix_alpha(F_alpha_constrained)
    F_beta_constrained = F_bb - Lambda_AO
    #environment.replace_current_fock_matrix_beta(F_beta_constrained)

    environment.replace_current_fock_matrices(F_alpha_constrained, F_beta_constrained)

constrain_step_thesis = gqcpy.FunctionalStep_UHFSCFEnvironment_d(constrain_function_thesis, description="Replace the alpha- and beta- UHF Fock matrices by their constrained counterparts, as explained in the CUHF 'thesis' algorithm.")

We'll now modify the plain UHF SCF algorithm with this modification `constrain_step_thesis`. From the algorithmic description printed above, we'll have to insert the Fock matrix modification at index `2`.

In [36]:
CUHF_algorithm_thesis = gqcpy.UHFSCFSolver_d.Plain(threshold = 1.0e-04, maximum_number_of_iterations = 1000)

CUHF_algorithm_thesis.insert(constrain_step_thesis, 2)
print(CUHF_algorithm_thesis.description())

An iterative algorithm (with a maximum of 1000 iterations) consisting of the following steps:
An algorithmic step consisting of 5 algorithmic steps:
	1. Calculate the current UHF alpha and beta density matrices (in the AO basis) and place them in the environment.
	2. Calculate the current UHF Fock matrices (expressed in the scalar/AO basis) and place them in the environment.
	3. Replace the alpha- and beta- UHF Fock matrices by their constrained counterparts, as explained in the CUHF 'thesis' algorithm.
	4. Solve the generalized eigenvalue problem for the most recent scalar/AO Fock matrices. Add the associated coefficient matrices and orbital energies to the environment.
	5. Calculate the current electronic UHF energy and place it in the environment.

With the following convergence criterion:
A convergence criterion that checks if the norm of the difference of two iterates (the UHF spin resolved density matrix in AO basis) is converged, with a tolerance of 1.00e-04.


Since we've now created the `CUHF_algorithm_thesis`, which is still an UHF SCF solver, we can set up an environment and optimize the CUHF model parameters.

In [37]:
environment = gqcpy.UHFSCFEnvironment_d.WithCoreGuess(N_alpha, N_beta, sq_hamiltonian, S)

In [38]:
cuhf_qc_structure_thesis = gqcpy.UHF_d.optimize(CUHF_algorithm_thesis, environment)
cuhf_energy_thesis = cuhf_qc_structure_thesis.groundStateEnergy()
cuhf_parameters_thesis = cuhf_qc_structure_thesis.groundStateParameters()

In [43]:
print("CUHF electronic energy (thesis): {:.8f}\n".format(cuhf_energy_thesis))
print("CUHF C_alpha (thesis):\n{}\n".format(cuhf_parameters_thesis.expansion().alpha.matrix()))
print("CUHF C_beta (thesis):\n{}\n".format(cuhf_parameters_thesis.expansion().beta.matrix()))

CUHF electronic energy (thesis): -2.91570739

CUHF C_alpha (thesis):
[[-4.755511e-01 -1.124780e+00  7.206944e-05]
 [-3.746464e-01  5.982052e-01 -9.965289e-01]
 [-3.746065e-01  5.983161e-01  9.964773e-01]]

CUHF C_beta (thesis):
[[ 2.552798e-01 -1.829360e-05 -1.194199e+00]
 [ 4.805616e-01  9.965123e-01  5.170136e-01]
 [ 4.805658e-01 -9.964939e-01  5.170450e-01]]



## Dense CIS calculations

In order to do a dense CIS calculation, GQCP uses its own framework for mathematical optimization. We'll have to create:
1. a dense eigenvalue problem solver
1. an associated eigenvalue problem environment

In [44]:
onv_basis = gqcpy.SpinResolvedSelectedONVBasis.CIS(K//2, 2, 1, True)

In [45]:
solver = gqcpy.EigenproblemSolver.Dense_d()
environment = gqcpy.CIEnvironment.Dense(sq_hamiltonian, onv_basis)

The encompassing `QCMethod` is `CI` (for any type of configuration interaction), so we'll use that to find the optimized and associated wave function model: a `LinearExpansion`. Internally, `QCMethod`s return a `QCStructure`, which wraps energies and optimized parameters for the ground state and excited states.

In [46]:
qc_structure = gqcpy.CI(onv_basis).optimize(solver, environment)

In order to find the total energy, we have to account for the nuclear repulsion. In GQCP, we use different kinds of first-quantized operators, and `NuclearRepulsionOperator` is one of them.

In [47]:
for i in range(onv_basis.dimension()):
    energy = qc_structure.energy(i) #+ gqcpy.NuclearRepulsionOperator(molecule.nuclearFramework()).value()
    print(energy)

-4.763313701028364
4.67346295388103e-310
0.14874577750301904
2.4e-322
0.0
5e-324
0.14874577750301904
0.17535310929707495
2.37e-322
4e-322


In [49]:
spinor_basis.numberOfSpinors()

6